# Data Cleaning and transformation process for orders and orders_stage files
# Step by step approach
Loading Data: The code starts by loading two datasets: orders.csv and order_stages.csv. These datasets should contain relevant information about orders and their stages.

Merging Datasets: The datasets are merged on a common key (e.g., order_id) to create a comprehensive view of each order's lifecycle.

Time Calculations: The code calculates the time spent in preparation and delivery stages by converting time columns to datetime format and computing the differences.

Average Time Analysis: It computes the average time spent in each stage and prints the results.

Visualization: The distribution of preparation and delivery times is visualized using histograms to identify patterns.

Identifying Longest Times: The code identifies the orders with the longest preparation and delivery times, which can highlight areas for improvement.

Translating lat/lag into degrees for distnace calculation :

Recommendations: Finally, the code suggests potential strategies for reducing courier time based on the analysis.

LET'S BEGIN WITH LOADING THE DATA AND UNDERSTANDING ITS DATA TYPE AND STRUCTURE

In [2]:

# This code snippet loads two CSV files containing order data and order stages, then inspects and displays their structure and the first few rows.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the datasets
orders_df = pd.read_csv('orders.csv')
order_stages_df = pd.read_csv('order_stages.csv')

# Inspect the structure of orders_df
print("Orders DataFrame:")
print(orders_df.info())
print("\
First few rows of orders_df:")
print(orders_df.head())

print("\
" + "="*50 + "\
")

# Inspect the structure of order_stages_df
print("Order Stages DataFrame:")
print(order_stages_df.info())
print("\
First few rows of order_stages_df:")
print(order_stages_df.head())

Orders DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65584 entries, 0 to 65583
Data columns (total 12 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   task_id                          65584 non-null  int64  
 1   order_id                         65584 non-null  int64  
 2   customer_id                      65584 non-null  int64  
 3   restaurant_id                    65584 non-null  int64  
 4   city                             65584 non-null  object 
 5   restaurant_address_lng           65584 non-null  float64
 6   restaurant_address_lat           65584 non-null  float64
 7   delivery_address_lng             65584 non-null  float64
 8   delivery_address_lat             65584 non-null  float64
 9   order_state                      65584 non-null  object 
 10  order_promised_delivery          65584 non-null  object 
 11  restaurant_finished_preparation  65525 non-null  object 
dtype

The orders dataset contains 65,584 entries with 12 columns. The columns include task_id, order_id, customer_id, restaurant_id, city, restaurant and delivery address coordinates, order state, and timestamps for promised delivery and restaurant preparation completion.

The order_stages dataset contains 384,937 entries with 15 columns. The columns include log_id, city, order_id, courier_id, vehicle_type, order_stage, timestamps, courier location coordinates, estimated travel times, distances, and task_id.

NOW LETS START DATA CLEANING AND TRANSFORMATION PROCESS

In [3]:
# This code snippet cleans and transforms two DataFrames, 'orders_df' and 'order_stages_df', by converting timestamp columns to datetime, calculating preparation times, filling missing values, and providing basic statistics.


import pandas as pd
import numpy as np
from datetime import datetime

# Clean and transform orders_df
def clean_orders(df):
    # Convert timestamp columns to datetime
    df['order_promised_delivery'] = pd.to_datetime(df['order_promised_delivery'])
    df['restaurant_finished_preparation'] = pd.to_datetime(df['restaurant_finished_preparation'])
    
    # Calculate preparation time in minutes
    df['preparation_time'] = (df['restaurant_finished_preparation'] - df['order_promised_delivery']).dt.total_seconds() / 60
    
    # Remove any rows with negative preparation time (data inconsistency)
    df = df[df['preparation_time'] >= 0]
    
    return df

# Clean and transform order_stages_df
def clean_order_stages(df):
    # Convert timestamp columns to datetime
    df['order_stage_start'] = pd.to_datetime(df['order_stage_start'])
    df['updated_expected_delivery_time'] = pd.to_datetime(df['updated_expected_delivery_time'])
    
    # Fill NaN values in estimated travel times with median values
    df['estimated_travel_time_in_seconds_to_restaurant'].fillna(df['estimated_travel_time_in_seconds_to_restaurant'].median(), inplace=True)
    df['estimated_travel_time_in_seconds_to_eater'].fillna(df['estimated_travel_time_in_seconds_to_eater'].median(), inplace=True)
    
    # Convert vehicle_type to categorical
    df['vehicle_type'] = pd.Categorical(df['vehicle_type'])
    
    return df

# Apply cleaning functions
cleaned_orders = clean_orders(orders_df)
cleaned_order_stages = clean_order_stages(order_stages_df)

# Display info and sample of cleaned datasets
print("Cleaned Orders DataFrame:")
print(cleaned_orders.info())
print("\
Sample of cleaned orders:")
print(cleaned_orders.head())

print("\
" + "="*50 + "\
")

print("Cleaned Order Stages DataFrame:")
print(cleaned_order_stages.info())
print("\
Sample of cleaned order stages:")
print(cleaned_order_stages.head())

# Basic statistics of numerical columns
print("\
Orders DataFrame - Basic Statistics:")
print(cleaned_orders.describe())

print("\
Order Stages DataFrame - Basic Statistics:")
print(cleaned_order_stages.describe())

# Check for missing values
print("\
Missing values in Orders DataFrame:")
print(cleaned_orders.isnull().sum())

print("\
Missing values in Order Stages DataFrame:")
print(cleaned_order_stages.isnull().sum())

Cleaned Orders DataFrame:
<class 'pandas.core.frame.DataFrame'>
Index: 1802 entries, 18 to 65574
Data columns (total 13 columns):
 #   Column                           Non-Null Count  Dtype              
---  ------                           --------------  -----              
 0   task_id                          1802 non-null   int64              
 1   order_id                         1802 non-null   int64              
 2   customer_id                      1802 non-null   int64              
 3   restaurant_id                    1802 non-null   int64              
 4   city                             1802 non-null   object             
 5   restaurant_address_lng           1802 non-null   float64            
 6   restaurant_address_lat           1802 non-null   float64            
 7   delivery_address_lng             1802 non-null   float64            
 8   delivery_address_lat             1802 non-null   float64            
 9   order_state                      1802 non-null   ob

UPDATED Orders_csv file
 AFTER THE DATA CLEANING PROCESS NEW DATA SET HAS- This dataset now contains 1,802 entries with 13 columns. The preparation time has been calculated and added as a new column.
UPDATED orders_stages_csv file
This dataset retains its original 384,937 entries with 15 columns. Missing values in estimated travel times have been filled with median values, and the vehicle_type column has been converted to a categorical type.

Now, lets check for the with data cleaning by checking for missing values, duplicates, and ensuring the data types are appropriate. Let's start with these steps

In [4]:
# Check for missing values and duplicates in both dataframes

# Check for missing values
missing_values_orders = orders_df.isnull().sum()
missing_values_order_stages = order_stages_df.isnull().sum()

# Check for duplicates
duplicates_orders = orders_df.duplicated().sum()
duplicates_order_stages = order_stages_df.duplicated().sum()

# Display the results
print('Missing Values in Orders DataFrame:')
print(missing_values_orders)
print('\
Missing Values in Order Stages DataFrame:')
print(missing_values_order_stages)

print('\
Number of Duplicates in Orders DataFrame:', duplicates_orders)
print('Number of Duplicates in Order Stages DataFrame:', duplicates_order_stages)

Missing Values in Orders DataFrame:
task_id                             0
order_id                            0
customer_id                         0
restaurant_id                       0
city                                0
restaurant_address_lng              0
restaurant_address_lat              0
delivery_address_lng                0
delivery_address_lat                0
order_state                         0
order_promised_delivery             0
restaurant_finished_preparation    59
preparation_time                   59
dtype: int64
Missing Values in Order Stages DataFrame:
log_id                                                 0
city                                                   0
order_id                                               0
courier_id                                             0
vehicle_type                                           0
order_stage                                            0
order_stage_start                                      0
courier_location

Now , handling the  the missing values by filling or dropping them as appropriate. Let's proceed with that.

In [5]:
# Fill or drop missing values

# For Orders DataFrame, fill missing 'restaurant_finished_preparation' with a placeholder or drop
orders_df['restaurant_finished_preparation'].fillna('Unknown', inplace=True)

# For Order Stages DataFrame, fill missing values with appropriate placeholders or drop
order_stages_df['courier_location_lng_at_start'].fillna(0, inplace=True)
order_stages_df['courier_location_lat_at_start'].fillna(0, inplace=True)
order_stages_df['estimated_travel_time_in_seconds_to_restaurant'].fillna(order_stages_df['estimated_travel_time_in_seconds_to_restaurant'].mean(), inplace=True)
order_stages_df['estimated_travel_time_in_seconds_to_eater'].fillna(order_stages_df['estimated_travel_time_in_seconds_to_eater'].mean(), inplace=True)
order_stages_df['distance_courier_to_restaurant_address'].fillna(order_stages_df['distance_courier_to_restaurant_address'].mean(), inplace=True)
order_stages_df['distance_courier_to_customer_address'].fillna(order_stages_df['distance_courier_to_customer_address'].mean(), inplace=True)
order_stages_df['updated_expected_delivery_time'].fillna('Unknown', inplace=True)

print('Missing values have been addressed.')

C:\Users\HP\AppData\Local\Temp\ipykernel_4368\1998925278.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Unknown' has dtype incompatible with datetime64[ns, UTC], please explicitly cast to a compatible dtype first.
  orders_df['restaurant_finished_preparation'].fillna('Unknown', inplace=True)
C:\Users\HP\AppData\Local\Temp\ipykernel_4368\1998925278.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Unknown' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  order_stages_df['updated_expected_delivery_time'].fillna('Unknown', inplace=True)


Missing values have been addressed.


Here Converting the lattitude and longitude values of restaurant and delivery addresses to convert in degrees

In [6]:
import numpy as np

# Function to calculate the distance between two points using the haversine formula
def haversine(lon1, lat1, lon2, lat2):
    # Convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    
    # Haversine formula
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    r = 6371  # Radius of earth in kilometers. Use 3956 for miles
    return c * r

# Apply the haversine function to calculate the distance between restaurant and delivery address
orders_df['distance_km'] = orders_df.apply(lambda row: haversine(row['restaurant_address_lng'], row['restaurant_address_lat'],
                                                                row['delivery_address_lng'], row['delivery_address_lat']), axis=1)

print('Distance calculation using haversine formula has been completed.')
print(orders_df[['restaurant_address_lng', 'restaurant_address_lat', 'delivery_address_lng', 'delivery_address_lat', 'distance_km']].head())

Distance calculation using haversine formula has been completed.
   restaurant_address_lng  restaurant_address_lat  delivery_address_lng  \
0               26.074823               44.415090             26.014888   
1               26.054794               44.454021             26.093656   
2               26.049776               44.424410             26.020117   
3               26.143047               44.430457             26.126890   
4               26.074823               44.415090             26.030227   

   delivery_address_lat  distance_km  
0             44.428852     4.999733  
1             44.434069     3.799977  
2             44.411906     2.735283  
3             44.417530     1.926774  
4             44.433221     4.075141  
